In [2]:
### loading libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import os
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

Matplotlib created a temporary config/cache directory at /scratch/local/u6031121/8997727/matplotlib-kuflqth4 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
# Check available devices
devices = tf.config.list_physical_devices('GPU')

if len(devices) > 0:
    print("GPU is available")
    for device in devices:
        print("Device name:", device.name)
else:
    print("No GPU is available")

GPU is available
Device name: /physical_device:GPU:0


## Training data

In [3]:
############################ TRAINING DATA 


In [4]:
########### Training Data
#Set the directory paths for each type of CSV file
directory_path_channel1 = '/uufs/chpc.utah.edu/common/home/gompert-group4/projects/fluctCNN/DataSepFilesNov23/EnvFiles/trainEnv/'
directory_path_channel2 = '/uufs/chpc.utah.edu/common/home/gompert-group4/projects/fluctCNN/DataSepFilesNov23/FreqFiles/trainFreq/'

# Get lists of all CSV files in each directory
csv_files_channel1 = [file for file in os.listdir(directory_path_channel1) if file.endswith('.csv')]
csv_files_channel2 = [file for file in os.listdir(directory_path_channel2) if file.endswith('.csv')]

# Initialize lists to store matrices and labels
# Process each pair of CSV files
matrices_channel1 = []
matrices_channel2 = []
out_vec1 = []

for file_channel1, file_channel2 in zip(csv_files_channel1, csv_files_channel2):
    # Read CSV file for channel 1
    df_channel1 = pd.read_csv(os.path.join(directory_path_channel1, file_channel1), header=None, sep=',', dtype='float32')
    matrices_channel1.append(df_channel1)
    
    # Read CSV file for channel 2
    df_channel2 = pd.read_csv(os.path.join(directory_path_channel2, file_channel2), header=None, sep=',', dtype='float32')
    matrices_channel2.append(df_channel2)
    
    # Appending first eight letters to out_vec as labels (adjust as needed)
    label = file_channel2[:9]
    out_vec1.append(label)
# Stack matrices for each channel
input_arrays_channel1 = np.stack(matrices_channel1)
input_arrays_channel2 = np.stack(matrices_channel2)
# Reshape the input arrays
inDa_trn_channel1 = input_arrays_channel1.reshape((len(csv_files_channel1), 10, 10))
inDa_trn_channel2 = input_arrays_channel2.reshape((len(csv_files_channel2), 10, 10))


# Stack the channels
inDa_trn = np.stack([inDa_trn_channel1, inDa_trn_channel2], axis=-1)

# Print the shape of the final input array
print(inDa_trn.shape)

(15000, 10, 10, 2)


In [5]:
######### TRAINIG labels
labels_train =out_vec1
# converting string values to integer calsses
from sklearn.preprocessing import LabelEncoder
# Initialize a LabelEncoder object
label_encoder = LabelEncoder()
# Fit the LabelEncoder to your string list and transform the string values into integer classes
label_train_int = label_encoder.fit_transform(labels_train)
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
ouLa_trn =to_categorical(label_train_int)
ouLa_trn.shape
#print(ouLa_trn)
ouLa_trn.shape
labelint_train = label_train_int
labelint_train.shape

(15000,)

## Test data

In [8]:
############################ TEST DATA 
# changing directory to the data folder
########### Training Data
#Set the directory paths for each type of CSV file
directory_path_tst_channel1 = '/uufs/chpc.utah.edu/common/home/gompert-group4/projects/fluctCNN/DataSepFilesNov23/EnvFiles/testEnv/'
directory_path_tst_channel2 = '/uufs/chpc.utah.edu/common/home/gompert-group4/projects/fluctCNN/DataSepFilesNov23/FreqFiles/testFreq/'

# Get lists of all CSV files in each directory
csv_files_tst_channel1 = [file for file in os.listdir(directory_path_tst_channel1) if file.endswith('.csv')]
csv_files_tst_channel2 = [file for file in os.listdir(directory_path_tst_channel2) if file.endswith('.csv')]

# Initialize lists to store matrices and labels
# Process each pair of CSV files
matrices_tst_channel1 = []
matrices_tst_channel2 = []
out_vec2 = []

for file_channel1, file_channel2 in zip(csv_files_tst_channel1, csv_files_tst_channel2):
    # Read CSV file for channel 1
    df_tst_channel1 = pd.read_csv(os.path.join(directory_path_tst_channel1, file_channel1), header=None, sep=',', dtype='float32')
    matrices_tst_channel1.append(df_tst_channel1)
    
    # Read CSV file for channel 2
    df_tst_channel2 = pd.read_csv(os.path.join(directory_path_tst_channel2, file_channel2), header=None, sep=',', dtype='float32')
    matrices_tst_channel2.append(df_tst_channel2)
    
    # Appending first eight letters to out_vec as labels (adjust as needed)
    label = file_channel2[:9]
    out_vec2.append(label)
# Stack matrices for each channel
input_arrays_tst_channel1 = np.stack(matrices_tst_channel1)
input_arrays_tst_channel2 = np.stack(matrices_tst_channel2)
# Reshape the input arrays
inDa_tst_channel1 = input_arrays_tst_channel1.reshape((len(csv_files_tst_channel1), 10, 10))
inDa_tst_channel2 = input_arrays_tst_channel2.reshape((len(csv_files_tst_channel2), 10, 10))


# Stack the channels
inDa_tst = np.stack([inDa_tst_channel1, inDa_tst_channel2], axis=-1)

# Print the shape of the final input array
print(inDa_tst.shape)

(3000, 10, 10, 2)


In [11]:
######### TEST labels

labels_tst =out_vec2
# converting string values to integer calsses
from sklearn.preprocessing import LabelEncoder
# Initialize a LabelEncoder object
label_encoder = LabelEncoder()
# Fit the LabelEncoder to your string list and transform the string values into integer classes
label_tst_int = label_encoder.fit_transform(labels_tst)
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
ouLa_tst =to_categorical(label_tst_int)
ouLa_tst.shape
#print(ouLa_trn)
ouLa_tst.shape
labelint_tst = label_tst_int
labelint_tst.shape

(3000,)

## Validation data

#### Independent validation set generated afresh from the simulation script

In [25]:
############################ VALIDATION DATA 

# changing directory to the data folder
########### Training Data
#Set the directory paths for each type of CSV file
directory_path_val_channel1 = '/uufs/chpc.utah.edu/common/home/gompert-group4/projects/fluctCNN/DataSepFilesNov23/EnvFiles/valEnv/'
directory_path_val_channel2 = '/uufs/chpc.utah.edu/common/home/gompert-group4/projects/fluctCNN/DataSepFilesNov23/FreqFiles/valFreq/'

# Get lists of all CSV files in each directory
csv_files_val_channel1 = [file for file in os.listdir(directory_path_val_channel1) if file.endswith('.csv')]
csv_files_val_channel2 = [file for file in os.listdir(directory_path_val_channel2) if file.endswith('.csv')]

# Initialize lists to store matrices and labels
# Process each pair of CSV files
matrices_val_channel1 = []
matrices_val_channel2 = []
out_vec3 = []

for file_channel1, file_channel2 in zip(csv_files_val_channel1, csv_files_val_channel2):
    # Read CSV file for channel 1
    df_val_channel1 = pd.read_csv(os.path.join(directory_path_val_channel1, file_channel1), header=None, sep=',', dtype='float32')
    matrices_val_channel1.append(df_val_channel1)
    
    # Read CSV file for channel 2
    df_val_channel2 = pd.read_csv(os.path.join(directory_path_val_channel2, file_channel2), header=None, sep=',', dtype='float32')
    matrices_val_channel2.append(df_val_channel2)
    
    # Appending first eight letters to out_vec as labels (adjust as needed)
    label = file_channel2[:9]
    out_vec2.append(label)
# Stack matrices for each channel
input_arrays_val_channel1 = np.stack(matrices_val_channel1)
input_arrays_val_channel2 = np.stack(matrices_val_channel2)
# Reshape the input arrays
inDa_val_channel1 = input_arrays_val_channel1.reshape((len(csv_files_val_channel1), 10, 10))
inDa_val_channel2 = input_arrays_val_channel2.reshape((len(csv_files_val_channel2), 10, 10))


# Stack the channels
inDa_val = np.stack([inDa_val_channel1, inDa_val_channel2], axis=-1)

# Print the shape of the final input array
print(inDa_val.shape)

(3000, 10, 10, 2)


In [28]:
labels_val =out_vec3
# converting string values to integer calsses
from sklearn.preprocessing import LabelEncoder
# Initialize a LabelEncoder object
label_encoder = LabelEncoder()
# Fit the LabelEncoder to your string list and transform the string values into integer classes
label_val_int = label_encoder.fit_transform(labels_val)

In [26]:
######### VALIDATION labels
labels_val =out_vec3
# converting string values to integer calsses
from sklearn.preprocessing import LabelEncoder
# Initialize a LabelEncoder object
label_encoder = LabelEncoder()
# Fit the LabelEncoder to your string list and transform the string values into integer classes
label_val_int = label_encoder.fit_transform(labels_val)
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
ouLa_val =to_categorical(label_val_int)
ouLa_val.shape
#print(ouLa_trn)
ouLa_val.shape
labelint_val = label_val_int
labelint_val.shape

ValueError: zero-size array to reduction operation maximum which has no identity

# # Machine Learning Algorithms

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import datasets
from sklearn import metrics

In [12]:
# reshaping arrays for randomforest
X_flattened_train = inDa_trn.reshape((inDa_trn.shape[0], -1))
X_flattened_tst = inDa_tst.reshape((inDa_tst.shape[0], -1))
X_flattened_train.shape

(45000, 200)

In [70]:
# Step 3: Create and train the Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42, min_samples_split=4)
rf_classifier.fit(X_flattened_train, labelint_train)



RandomForestClassifier(min_samples_split=4, n_estimators=200, random_state=42)

In [71]:
y_pred = rf_classifier.predict(X_flattened_tst)
accuracy = accuracy_score(labelint_test, y_pred)

In [72]:
print(f"Accuracy: {accuracy}")

print("Classification Report:\n", classification_report(labelint_test, y_pred))

Accuracy: 0.6373333333333333
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.89      0.94      5000
           1       0.48      0.48      0.48      5000
           2       0.48      0.54      0.51      5000

    accuracy                           0.64     15000
   macro avg       0.65      0.64      0.64     15000
weighted avg       0.65      0.64      0.64     15000



In [ ]:
# Additionally, you can visualize the confusion matrix
conf_matrix = metrics.confusion_matrix(labelint_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# Step 4: Make predictions on the test set
y_pred = rf_classifier.predict(inDa_tst)

# Step 5: Evaluate the model
accuracy = accuracy_score(ouLa_tst, y_pred)
print(f"Accuracy: {accuracy}")

# You can also print a classification report for more detailed metrics
print("Classification Report:\n", classification_report(ouLa_tst, y_pred))

# Additionally, you can visualize the confusion matrix
conf_matrix = metrics.confusion_matrix(ouLa_tst, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# XGBoost

In [73]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 142.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [64]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [7]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [13]:
dtrain = xgb.DMatrix(X_flattened_train, label=labelint_train)
dtest = xgb.DMatrix(X_flattened_tst, label=labelint_test)


In [18]:
# Set hyperparameters and train the model
params = {
    'objective': 'multi:softmax',  # Change to 'multi:softmax' for multiclass classification
    'eval_metric': 'logloss',         # Change based on your problem
    'max_depth': 3,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'num_class':3
}


In [19]:
num_round = 100  # Number of boosting rounds

model = xgb.train(params, dtrain, num_round)

# Predictions on the test set
y_pred = model.predict(dtest)

/uufs/chpc.utah.edu/common/home/u6031121/.local/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:42:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


In [22]:
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
accuracy = accuracy_score(labelint_test, y_pred_binary)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(labelint_test, y_pred_binary))

Accuracy: 0.5593333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.75      0.81      5000
           1       0.43      0.93      0.59      5000
           2       0.00      0.00      0.00      5000

    accuracy                           0.56     15000
   macro avg       0.44      0.56      0.47     15000
weighted avg       0.44      0.56      0.47     15000



/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# CNN model

In [38]:
### CNN

model = keras.Sequential()
model.add(layers.Conv2D(64, kernel_size=(3, 3),padding = "same", activation="relu", input_shape=(10,20,1)))
model.add(layers.Conv2D(256, kernel_size=(3, 3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(3, activation="softmax"))


optimizer = keras.optimizers.Adam(learning_rate=0.0001,
    beta_1=0.8,
    beta_2=0.88,
    #momentum=0.4,
    epsilon=1e-07,)

# Compile the model
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
es = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)
history = model.fit(inDa_trn, ouLa_trn, batch_size=96, epochs=35,validation_data=(inDa_tst, ouLa_tst), callbacks=[es] )
print(history.history.keys())

# Evaluate the model on the test set
#loss, accuracy = model.evaluate(inDa_val, ouLa_val)
# print("Test Loss:", loss)
# print("Test Accuracy:", accuracy)

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Epoch 1/35


ValueError: in user code:

    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /opt/conda/lib/python3.9/site-pack

In [ ]:
# Making confusion matrix to find percentage of false positives an dfalse negatives
# use of test data to make predictions based on the model fitted
lable_predicted = model.predict(inDa_val)
lable_predicted_categories = np.argmax(lable_predicted, axis=1)
label_actual = np.argmax(ouLa_val, axis =1)

In [ ]:
# calculaiton of confusion matrix\
cm = confusion_matrix(label_actual, lable_predicted_categories)
print(cm)
type(cm)


In [ ]:
# iterating through numpy array to convert the values into percentage

oneD_total = np.sum(cm, axis =1)
cm_perc = cm/oneD_total[:,np.newaxis]
cm_perc

In [ ]:
class_names = [0,1,2] # the vector defines three classes of selection types

In [ ]:
# graphical visualization of the confusion matrix in the form of heatmap
plt.figure(figsize=(8,6))
sns.heatmap(cm_perc,  annot=True, cmap ='Blues', xticklabels =  class_names, yticklabels = class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
array_2d = np.array([[1, 2, 3],
                     [4, 5, 6],
                     [7, 8, 9]])

# Calculate the sum of each 1D array (row)
row_sums = np.sum(array_2d, axis=1)
row_sums